
Developed by Abhijeet Mote
+1(818) 456-1013

abhijeetmote@gmail.com
Following code is written in Jypyter Notebook because presentable, this can be written as script also. With Object Oriented fashion easily.
This code can be improved in terms of :
* readbiility, 
* PEP8 standards 
* SOLID Design principles 
* unit Testcases


Kindly provide the window_size before running the code default is 0


In [21]:
# window_size in days, 0 being same day and 3 being 3 day window

window_size = 0

if window_size == 0 :
    window_size = 1
print(f"window_size : {window_size}")

window_size : 1


In [2]:
# written code according to python3.8
import sys
sys.version_info

sys.version_info(major=3, minor=8, micro=8, releaselevel='final', serial=0)

In [3]:
# import required libraries 

import pandas as pd
import datetime
import zipfile

pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_colwidth', 100)  # or 199

In [4]:
# increase the width of cell

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
def dt_inplace(df: pd.DataFrame) -> pd.DataFrame:
    """Automatically detect and convert (in place!) each
    dataframe column of datatype 'object' to a datetime
    and returns the resulting dataframe
    """
    from pandas.errors import ParserError
    for c in df.columns[df.dtypes=='object']: #don't cnvt num
        try:
            df[c]=pd.to_datetime(df[c])
        except (ParserError,ValueError): #Can't cnvrt some
            pass # ...so leave whole column as-is unconverted
    return df

def read_csv(*args:str , **kwargs:str) -> pd.DataFrame:
    """Drop-in replacement for Pandas pd.read_csv. It invokes
    pd.read_csv() (passing its arguments) and then auto-
    matically detects and converts each column whose datatype
    is 'object' to a datetime and returns the resulting dataframe.
    """
    return dt_inplace(pd.read_csv(*args, **kwargs))


def unzip(path_to_zip_file:str, directory_to_extract_to:str)->None:
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)


In [16]:
# unzip the zip file 
try:
    unzip("data/transactions_medium.zip", "data")
except Exception as e:
    print("Error occured, ",e)

In [8]:
# read the data given transaction data 
transactions_df = read_csv("data/transactions_medium.csv",sep= "|")

# sorting data based on TIMESTAMP 
transactions_df.sort_values(['TIMESTAMP'],inplace=True, ignore_index=True)


# Assumption

### there are few data transactions where sender and receiver is same 

In [9]:
# analyse same sender an receiver 
same_sender_receiver_df = pd.read_csv("same_sender_receiver.csv")
same_sender_receiver_df[["TRANSACTION_x","TIMESTAMP_x","AMOUNT_x","SENDER_x","RECEIVER_x", "SENDER_y","RECEIVER_y"]].head()

,TRANSACTION_x,TIMESTAMP_x,AMOUNT_x,SENDER_x,RECEIVER_x,SENDER_y,RECEIVER_y
0,f7fa68c9abea4c7dcc812245da827483a772ee42fa459279965409da1416b2e620100227,2010-02-27 10:55:12,160.0000,ID00000000000000,ID00000000000000,ID00000000000000,ID00000000000000
1,f7fa68c9abea4c7dcc812245da827483a772ee42fa459279965409da1416b2e620100227,2010-02-27 10:55:12,160.0000,ID00000000000000,ID00000000000000,ID00000000000000,ID00000000000000
2,f7fa68c9abea4c7dcc812245da827483a772ee42fa459279965409da1416b2e620100227,2010-02-27 10:55:12,160.0000,ID00000000000000,ID00000000000000,ID00000000000000,ID00000000000000
3,f7fa68c9abea4c7dcc812245da827483a772ee42fa459279965409da1416b2e620100227,2010-02-27 10:55:12,160.0000,ID00000000000000,ID00000000000000,ID00000000000000,ID00000000000000
4,f7fa68c9abea4c7dcc812245da827483a772ee42fa459279965409da1416b2e620100227,2010-02-27 10:55:12,160.0000,ID00000000000000,ID00000000000000,ID00000000000000,ID00000000000000


In [10]:
def data_cleaning(transactions_df: pd.DataFrame, window_size: int)-> pd.DataFrame:
    """Data Cleaning as per the requirements 
    data_cleaning accepts dataframe and 
    returns pd.DataFrame as per the requirment logi
    """
    
    final_df = pd.DataFrame()
    final_list = []
    
    for index, df in transactions_df[:].groupby(pd.Grouper(key='TIMESTAMP', axis=0, freq=f'{window_size}D')):
        
        # made inner join on grouped df to find out bridge
        merged_df = pd.merge(df, df, how='inner',left_on='RECEIVER', right_on = "SENDER")
                
        # dropped where sender and receiver is same, please see the assumption.
        clean_df = merged_df[merged_df.RECEIVER_x != merged_df.RECEIVER_y]
        
        # index reset done 
        clean_df.reset_index(drop=True, inplace=True)
        
        # checking if the bridges found then create new dataframe to accumulate that data 
        if len(clean_df) > 0:
            
            # to find the bridge count 
            total_bridges = 0
            for index, data in clean_df.iterrows():
                
                # check if the bridge_fee 0.01 to 0.10, inclusive is taken by the receiver or not ? 
                if (data.AMOUNT_x - data.AMOUNT_x * .01)  >= data.AMOUNT_y >= (data.AMOUNT_x - data.AMOUNT_x * 0.1):
                    total_bridges = total_bridges + 1
                    final_list.append(clean_df.iloc[index].tolist())
            else:
                # This block will only execute at the when iteration is done on df 
                
                tmp_df = pd.DataFrame(final_list)
                
                # assign new column and value of total bridges to those transactions 
                tmp_df["#Bridges"] = total_bridges
                # append all the final_df and tmp_df data to be outputed at the end
                final_df = final_df.append(tmp_df, ignore_index=True)
    
    # remove/ delete the df/varabiles which are not used for memory relase and faster processing.
    del tmp_df, clean_df, transactions_df, final_list, total_bridges
    return final_df


In [11]:
# call data_cleaning function 
e = datetime.datetime.now(); print("Processing start time:",datetime.datetime.now())
final_df = data_cleaning(transactions_df, window_size)
e = datetime.datetime.now(); print("Processing end time  :",datetime.datetime.now())
# name the data columns properly 
final_df.columns = ['#Bridges','TRANSACTION_x', 'TIMESTAMP_x', 'AMOUNT_x', 'SENDER_x', 'RECEIVER_x' ,'TRANSACTION_y', 'TIMESTAMP_y', 'AMOUNT_y', 'SENDER_y','RECEIVER_y',]
display(final_df)

Processing start time: 2022-07-04 20:54:08.338063
Processing end time : 2022-07-04 21:10:01.825387


,#Bridges,TRANSACTION_x,TIMESTAMP_x,AMOUNT_x,SENDER_x,RECEIVER_x,TRANSACTION_y,TIMESTAMP_y,AMOUNT_y,SENDER_y,RECEIVER_y
0,72,a63cae94c4743d8ebaf2cbf3b864246480a2892d0672f95f9fcf582985a1efa820100303,2010-03-03 10:55:12,16158.0000,ID00001692270706,ID00001692357908,87e24e2bc62f2bdfc86e82bbd277a98a0f28830c196252833fcdd2a80c622f0820100303,2010-03-03 10:55:12,15121.8800,ID00001692357908,ID00001693727507
1,72,dc3c09cd982710f9109ea8dcaa12c9a2b4ba88a15d665dee19e75ec14deb649020100303,2010-03-03 10:55:12,6648.3100,ID00001692270706,ID00001692357908,6684b0fc3a8a26147301f6e0036dfa17817a477cfc8e29a00fbaade43a8dba9720100303,2010-03-03 10:55:12,6043.9200,ID00001692357908,ID00001693726339
2,72,dc3c09cd982710f9109ea8dcaa12c9a2b4ba88a15d665dee19e75ec14deb649020100303,2010-03-03 10:55:12,6648.3100,ID00001692270706,ID00001692357908,ce8391234a70ea84762049313258316b9539ae53e85ddca9f7ec009185a6cb6d20100303,2010-03-03 10:55:12,6043.9200,ID00001692357908,ID00001693727262
3,72,6fd68ed5c15ddebba97088a6d93536faaeb7138c3fceafd65fec6a008e76a04220100303,2010-03-03 10:55:12,6648.3100,ID00001692270706,ID00001692357908,6684b0fc3a8a26147301f6e0036dfa17817a477cfc8e29a00fbaade43a8dba9720100303,2010-03-03 10:55:12,6043.9200,ID00001692357908,ID00001693726339
4,72,6fd68ed5c15ddebba97088a6d93536faaeb7138c3fceafd65fec6a008e76a04220100303,2010-03-03 10:55:12,6648.3100,ID00001692270706,ID00001692357908,ce8391234a70ea84762049313258316b9539ae53e85ddca9f7ec009185a6cb6d20100303,2010-03-03 10:55:12,6043.9200,ID00001692357908,ID00001693727262
...,...,...,...,...,...,...,...,...,...,...,...
5771632,11,87ccab650d9a9233e7cb726b83da9ed6ee6f5e4206272bcaddd123d74a27c37d20111231,2011-12-31 10:55:12,441956.0500,ID00001692307437,ID00001692307431,30cc1c1d08273b2f050a403a75098163cfde97245e7ab957fc798add6edf502220111231,2011-12-31 10:55:12,414000.0000,ID00001692307431,ID00001692307432
5771633,11,5d4e4370aaef16227d629582066f9f820dc9d5cc94de0c6125a531156d13738a20111231,2011-12-31 10:55:12,1632000.0000,ID00001692488287,ID00001692307432,ef04c8bd90a2f2be4ba677cede91bacaba869b0e7574bffb504640b839dc4f3820111231,2011-12-31 10:55:12,1505000.0000,ID00001692307432,ID00001692308087
5771634,11,df8ecc50a2a28905893bb8850d6b3c34e269f19517bcb991dcb320d3996f442020111231,2011-12-31 10:55:12,1606648.8900,ID00001692488287,ID00001692307432,ef04c8bd90a2f2be4ba677cede91bacaba869b0e7574bffb504640b839dc4f3820111231,2011-12-31 10:55:12,1505000.0000,ID00001692307432,ID00001692308087
5771635,11,8e01bcddfea010fff5bc9c2bc575cd2da222937b7c52e3528c8bb4eba28e17b420111231,2011-12-31 10:55:12,9595.8300,ID00001692272382,ID00001692270706,f4d00424686b7681ccad83d934207f349d13ff975fcf605f522643879bc6e49220111231,2011-12-31 10:55:12,8796.6700,ID00001692270706,ID00001692189152


In [12]:
# group the bridge persons and sort it based on count of bridges in descending(highest to lowest) order 
report_df = final_df.groupby(["SENDER_y"]).sum().sort_values(['#Bridges'],ascending=False).reset_index()


In [13]:
# rank the bride persons 
report_df["Rank"] = range(1, len(report_df)+1)

In [14]:
# rename the dataframe 
report_df.rename(columns = {'SENDER_y':'Top_Senders', 'AMOUNT_y':'Total$'}, inplace = True)

# display the top 10 Bridges with total value 
print(report_df[['Rank', 'Top_Senders', '#Bridges',  'Total$']].head(10).to_string(index=False))

 Rank      Top_Senders  #Bridges             Total$
    1 ID00001692186081  56072819      41606614.0800
    2 ID00001692199091  32938194 2520083947100.1382
    3 ID00001692189152  27318742 3420375706740.2266
    4 ID00001692209270  19335675  768182598270.5050
    5 ID00001692270706  15963174    8920147038.4303
    6 ID00001692307432  14549753   92444930408.4897
    7 ID00001692306657  12006071   21559506779.5923
    8 ID00001692190076   9724413   43865832967.2005
    9 ID00001692198772   5211946  355579489238.2180
   10 ID00001692270601   5034274   10729329228.3894


# Final thoughts 


# My Machine's configuration : Core i7 4 cores, 16GB RAM 

# Time and memory complexities

     Time Complexity : O(n*n) 
     
     Memory Complexity : will be O(n) or linear where n is the number of elements in our dataframe/list
     
# Data Processing time : ~14 mins 

# Final Report

In [15]:
print(report_df[['Rank', 'Top_Senders', '#Bridges',  'Total$']].head(10).to_string(index=False))

 Rank      Top_Senders  #Bridges             Total$
    1 ID00001692186081  56072819      41606614.0800
    2 ID00001692199091  32938194 2520083947100.1382
    3 ID00001692189152  27318742 3420375706740.2266
    4 ID00001692209270  19335675  768182598270.5050
    5 ID00001692270706  15963174    8920147038.4303
    6 ID00001692307432  14549753   92444930408.4897
    7 ID00001692306657  12006071   21559506779.5923
    8 ID00001692190076   9724413   43865832967.2005
    9 ID00001692198772   5211946  355579489238.2180
   10 ID00001692270601   5034274   10729329228.3894


# Good to explore about this and can be created Parallel tasks 

https://orion-docs.prefect.io/tutorials/first-steps/